# 1. Code của chương trình. Khi sử dụng code thì "Run All" và thao tác ở task "Chạy chương trình"

In [36]:
import pandas as pd
import numpy as np
import math

In [37]:
def doc_input(ten_file):
    # Read data from the Excel file "input.xlsx"
    df = pd.read_excel(ten_file, header=None)
    
    # Extract x values from the first row (index 0)
    x = df.iloc[0].values
    x = np.array(x, dtype=float)
    
    # Extract y values from the second row (index 1)
    y = df.iloc[1].values
    y = np.array(y, dtype=float)
    
    return x, y
    
def cacKhoangDonDieuChuaNghiem(x, y, y_):
    listInvervals = []
    iStart = 0
    iStop = 0
    for i in range(1, len(x) - 1):
        sign = np.sign(y[i] - y[i - 1])
        if sign != np.sign(y[i + 1] - y[i]):
            iStop = i
            listInvervals.append([x[iStart:iStop + 1], y[iStart:iStop + 1]])
            iStart = iStop
    listInvervals.append([x[iStart:len(x)], y[iStart:len(y)]])
    listResult = []
    for i in listInvervals:
        if y_ > min(i[1]) and y_ < max(i[1]):
            listResult.append([i[0], i[1]])
    return listResult

## Phương pháp hàm ngược

In [38]:
def hoocnerNhan(x):
    a = [1, -x[0]]
    for i in range(1, len(x)):
        a.append(0)
        temp = a.copy()
        for j in range(1, len(a)):
            a[j] = temp[j] - temp[j - 1] * x[i]
        a[0] = 1
    return a
    
def hoocnerChia(a, value):
    b = [a[0]]
    for i in range(1, len(a)):
        b.append(a[i] + b[i - 1] * value)
    return b

def noiSuyLagrange(x, y):
    a = np.zeros(len(x))
    phi = hoocnerNhan(x)
    for i in range(len(x)):
        omega = hoocnerChia(phi, x[i])
        omega.pop()
        k = y[i]/hoocnerChia(omega, x[i]).pop()
        for j in range(len(a)):
            a[j] += k * omega[j]
    return a.tolist()

def hamNguoc(x, y, y_):
    return hoocnerChia(noiSuyLagrange(y, x), y_).pop()

## Phương pháp lặp Newton 

## Phương pháp Newton tiến và Newton lùi cho lặp Newton

In [39]:
def hoocnerNhanBoSung(coeffTich, t):
    if len(coeffTich) == 0:
        coeffTich.append(1)
        coeffTich.append(-t)
    else:
        coeffTich.append(0)
        temp = coeffTich.copy()
        for i in range(1, len(coeffTich)):
            coeffTich[i] = coeffTich[i] - t * temp[i - 1]
    return coeffTich

# Nội suy Newton tiến
def saiPhanTien(delta, yk):
    temp = delta.copy()
    delta.append(0)
    delta[0] = yk
    for i in range(1, len(delta)):
        delta[i] = delta[i - 1] - temp[i - 1]
    return delta  

def ns_NewtonTien(x, y):
    a = [y[0]]
    delta = [y[0]]
    coeffTich = []
    giaiThua_k = 1
    for i in range(1, len(x)):
        delta = saiPhanTien(delta, y[i])
        coeffTich = hoocnerNhanBoSung(coeffTich, i - 1)    
        giaiThua_k *= i
        c = delta[len(delta) - 1]/ giaiThua_k
        a_1 = a.copy()
        a.append(0)
        for j in range(1, len(a)):
            a[j] = a_1[j - 1] + c * coeffTich[j]
        a[0] = c
    return a

# Nội suy Newton lùi
def saiPhanLui(lamda, yk):
    temp = lamda.copy()
    lamda.append(0)
    lamda[0] = yk
    for i in range(1, len(lamda)):
        lamda[i] = temp[i - 1] - lamda[i - 1]
    return lamda

def ns_NewtonLui(x, y):
    a = [y[len(y) - 1]]
    lamda = [y[len(y) - 1]]
    coeffTich = []
    giaiThua_k = 1
    for i in range(1, len(x)):
        lamda = saiPhanLui(lamda, y[len(y) - 1 - i])
        coeffTich = hoocnerNhanBoSung(coeffTich, 1 - i)    
        giaiThua_k *= i
        c = lamda[len(lamda) - 1]/ giaiThua_k
        a_1 = a.copy()
        a.append(0)
        for j in range(1, len(a)):
            a[j] = a_1[j - 1] + c * coeffTich[j]
        a[0] = c
    return a

## Lặp Newton

In [40]:
def lapNewton(x, y, y_, index, epsi):
    #index chỉ số của mốc bên phải y_ là yk+1: y_nằm giữa 2 mốc yk và yk+1
    soLanLap = 1
    if index + 1 >= len(x) - index + 1:
        # Sử dụng newton lùi
        n_used = min(index + 1, 7)
        x_used = x[index + 1 - n_used:index + 1]
        y_used = y[index + 1 - n_used:index + 1]
        a = ns_NewtonLui(x_used, y_used)

        # Tìm hệ số của hàm phi theo đa thức nội suy
        laplaY = y_used[len(y_used) - 1] - y_used[len(y_used) - 2]
        for i in range(len(a) - 2):
            a[i] = -a[i]/laplaY
        a[len(a) - 2] = -(a[len(a) - 2] - laplaY)/laplaY
        a[len(a) - 1] = (y_ - y_used[len(y_used) - 1])/laplaY

        t0 = (y_ - y_used[len(y_used) - 1])/(y_used[len(y_used) - 1] - y_used[len(y_used) - 2])
        t1 = hoocnerChia(a, t0).pop()
        while (abs(t1 - t0)/abs(t1) >= epsi):
            t0 = t1
            t1 = hoocnerChia(a, t0).pop()
            soLanLap += 1
        print("Với khoảng chứa nghiệm sử dụng là: \nx = {0}\ny = {1}".format(x_used, y_used))
        return [t1 * (x[1] - x[0]) + x_used[len(x_used) - 1], soLanLap, 0]
    else:
        # Sử dụng newton tiến
        n_used = min(len(x) - index + 1, 7)
        x_used = x[index - 1:index + n_used - 1]
        y_used = y[index - 1:index + n_used - 1]
        a = ns_NewtonTien(x_used, y_used)

        # tìm hệ số của đa thức phi(t) dựa vào đa thức nội suy newton
        deltaY = y_used[1] - y_used[0]
        for i in range(len(a) - 2):
            a[i] = -a[i]/deltaY
        a[len(a) - 2] = -(a[len(a) - 2] - deltaY)/deltaY
        a[len(a) - 1] = (y_ - y_used[0])/deltaY

        t0 = (y_ - y_used[0])/deltaY
        t1 = hoocnerChia(a, t0).pop()
        while (abs(t1 - t0)/abs(t1) >= epsi):
            t0 = t1
            t1 = hoocnerChia(a, t0).pop()
            soLanLap += 1
        print("Với khoảng chứa nghiệm sử dụng là: \nx = {0}\ny = {1}".format(x_used, y_used))
        return [t1 * (x[1] - x[0]) + x_used[0], soLanLap, 1]

## Nội suy ngược

In [41]:
def noiSuyNguoc(x, y, y_, epsi = 1e-10):
    solutionsIntervals = cacKhoangDonDieuChuaNghiem(x, y, y_)
    solutions = []
    t = 10
    for i in solutionsIntervals:
        # Tìm vị trí của mốc bên phải y_: yk+1
        for j in range(len(i[1]) - 1):
                if (i[1][j] - y_)*(i[1][j + 1] - y_) < 0:
                    index = j + 1

        # Phương pháp hàm ngược thì chọn t mốc cạnh yk+1z
        if len(i[1]) >= t:
            if index >= t:
                x_used = i[0][index - t+1:index + 1]
                y_used = i[1][index - t+1:index + 1]
            else:
                x_used = i[0][0:t]
                y_used = i[1][0:t]
            root = hamNguoc(x_used, y_used, y_)
            solutions.append(root)
            print("Phuong phap su dung: ham nguoc")
            print("Voi khoang chua nghiem: \nx = {0}\ny = {1}\nCo nghiem la: {2}\n".format(x_used, y_used, root))

        # Phương pháp lặp thì chọn min(7, số mốc có thể sử dụng)
        else:
            for j in range(len(x)):
                if i[0][index] == x[j]:
                    indexInData = j
            result = lapNewton(x, y, y_, indexInData, epsi)
            solutions.append(result[0])
            if result[2] == 0:
                print("Phuong phap su dung: lap Newton lui")
            else:
                print("Phuong phap su dung: lap Newton tien")
            print("Nghiem la: {0}\nSo lan lap: {1}\n".format(result[0], result[1]))
    return solutions

# 2. Chạy chương trình

In [42]:
x, y = doc_input("inputNSNguoc.xlsx")

# Giá trị y_ = f(x_): tìm nghiệm x_ để f(x_) = y_
y_ = 1
solutions = noiSuyNguoc(x, y, y_)

Phuong phap su dung: ham nguoc
Voi khoang chua nghiem: 
x = [3.242 3.36  3.478 3.596 3.714 3.832 3.95  4.068 4.186 4.304]
y = [4.1263 3.8493 3.5373 3.1948 2.8267 2.4384 2.0354 1.6234 1.2084 0.7962]
Co nghiem la: 4.245432011978246

